In [ ]:
import pandas as pd
import warnings

# Ignorar o warning de SettingWithCopy
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Carregar os dados
file_path = r"C:/Kidy/PREDITIVA\DADOS_PREDITIVA.xlsx"  # ajuste o caminho conforme necessário
df = pd.read_excel(file_path, sheet_name='DADOS PREDITIVA')

# Função principal
def analisar_cliente(codigo_cliente=None, codigo_grupo_cliente=None):
    # Filtro por cliente ou grupo de cliente
    if codigo_cliente:
        dados_cliente = df[df['Codigo Cliente'] == codigo_cliente].copy()
    elif codigo_grupo_cliente:
        dados_cliente = df[df['Codigo Grupo Cliente'] == codigo_grupo_cliente].copy()
    else:
        print("Informe um código de cliente ou código de grupo de cliente!")
        return

    if dados_cliente.empty:
        print("Nenhum dado encontrado para o código informado!")
        return
    
    # Pegar o nome do cliente (ou grupo) e data da última compra
    nome_cliente = dados_cliente['Razao Social'].iloc[0] if codigo_cliente else dados_cliente['Grupo Cliente'].iloc[0]
    ultima_compra = dados_cliente['Data Ultima Compra'].max().strftime('%d/%m/%Y')
    
    print("==========================")
    print(f"📌 Cliente: {nome_cliente}")
    print(f"🗓️  Última Compra: {ultima_compra}")
    print("==========================")
    
    # Criar uma coluna com o mês da data de cadastro
    dados_cliente.loc[:, 'Mes Cadastro'] = dados_cliente['Data Cadastro'].dt.month
    
    # Analisar as datas de TODOS os pedidos (não só o cadastro)
    dados_cliente.loc[:, 'Mes Pedido'] = dados_cliente['Data Cadastro'].dt.month  # se quiser considerar outra data, ajustar aqui
    
    # Melhor mês com base na frequência dos pedidos realizados
    melhor_mes = dados_cliente['Mes Pedido'].mode()[0]
    
    print(f"\n👉 Melhor mês para oferecer produtos: {melhor_mes} (Baseado nas datas dos pedidos cadastrados)")

    # Análise das 10 melhores linhas de produto
    top_linhas = (dados_cliente.groupby('Linha')['Qtd Venda']
                  .sum()
                  .sort_values(ascending=False)
                  .head(10))
    
    print("\n👉 Top 10 Linhas de Produtos para Oferecer:")
    for idx, (linha, qtd) in enumerate(top_linhas.items(), 1):
        print(f"{idx}. {linha} - {qtd} unidades vendidas")

# Exemplo de uso:
# Digite o código do cliente ou do grupo de cliente
# analisar_cliente(codigo_cliente=10021)
# analisar_cliente(codigo_grupo_cliente='NÃO INFORMADO')

# Teste manual
analisar_cliente(codigo_cliente=1075)
